In [57]:
import duckdb
import pandas as pd

## Creating datasets

In [58]:

original_data = pd.DataFrame({
    'id' : [1, 2, 3, 4],
    'name' : ['Bob','Alice', 'Charlie', 'David'],
    'last_update' : pd.to_datetime(['2020-01-01','2021-01-01','2021-01-01','2021-01-01'])
})

new_data = pd.DataFrame({
    'id' : [2,3,4,5],
    'name' : ['Alice', 'Charlie', 'David', 'Eve'],
    'last_update' : pd.to_datetime(['2021-01-01','2021-01-01','2022-01-01','2022-01-01'])
})

In [59]:
# creating a DuckDB database
file = 'data.db'
con = duckdb.connect(file)

# adding the original data to the database
con.sql(("CREATE OR REPLACE TABLE original_data AS SELECT * FROM original_data"))

# show tables
print(con.sql('SHOW TABLES').fetchdf())

# show the original data
print(con.sql('SELECT * FROM original_data').fetchdf())

# close the connection
con.close()

            name
0  original_data
   id     name last_update
0   1      Bob  2020-01-01
1   2    Alice  2021-01-01
2   3  Charlie  2021-01-01
3   4    David  2021-01-01


## Append load method


In [ ]:
def append_load(con, data):
    # creating a copy of the original data (not necessary in general)
    con.sql(("CREATE OR REPLACE TABLE data_append AS SELECT * FROM original_data"))
    # getting max id
    max_id = con.sql('SELECT MAX(id) FROM data_append').fetchdf().values[0][0]
    # selecting the new data
    data = data[data['id'] > max_id]
    # appending the new data
    for _, row in data.iterrows():
        date_str = row['last_update'].strftime('%Y-%m-%d')
        con.sql(f"INSERT INTO data_append VALUES ({row['id']}, '{row['name']}', '{date_str}')")
    
with duckdb.connect(file) as con:
    con.sql('BEGIN TRANSACTION') # starting a transaction -- changes are synced once = improves performance
    append_load(con, new_data)
    con.sql('COMMIT') # committing the transaction
    print(con.sql('SELECT * FROM data_append').fetchdf())

   id     name last_update
0   1      Bob  2020-01-01
1   2    Alice  2021-01-01
2   3  Charlie  2021-01-01
3   4    David  2021-01-01
4   5      Eve  2022-01-01


## Incremental load method


In [ ]:
def incremental_load(con, data):
    # creating a copy of the original data (not necessary in general)
    con.sql(("CREATE OR REPLACE TABLE data_incremental AS SELECT * FROM original_data"))

    # deleting "outdated" rows
    new_earliest_date = data['last_update'].min().strftime('%Y-%m-%d')
    con.sql(f"DELETE FROM data_incremental WHERE last_update < '{new_earliest_date}'")


    # removing rows that will be updated
    ids_timestamps = con.sql('SELECT id, last_update FROM data_incremental').fetchdf()
    # merging new data to compoare last_update
    ids_timestamps = ids_timestamps.merge(data[['id','last_update']], on='id', 
                                          suffixes=('_old', '_new'))
    # finding ids for which the dates are different
    ids_to_remove = ids_timestamps[ids_timestamps['last_update_old'] != ids_timestamps['last_update_new']]['id']
    # removing rows
    for id in ids_to_remove:
        con.sql(f"DELETE FROM data_incremental WHERE id = {id}")


    # appending the new data
    most_recent_date = con.sql('SELECT MAX(last_update) FROM data_incremental').fetchdf().values[0][0]
    data = data[data['last_update'] > most_recent_date]
    for _, row in data.iterrows():
        date_str = row['last_update'].strftime('%Y-%m-%d')
        con.sql(f"INSERT INTO data_incremental VALUES ({row['id']}, '{row['name']}', '{date_str}')")

with duckdb.connect(file) as con:
    con.sql('BEGIN TRANSACTION') # starting a transaction -- changes are synced once = improves performance
    incremental_load(con, new_data)
    con.sql('COMMIT') # committing the transaction 
    print(con.sql('SELECT * FROM data_incremental').fetchdf())


   id     name last_update
0   2    Alice  2021-01-01
1   3  Charlie  2021-01-01
2   4    David  2022-01-01
3   5      Eve  2022-01-01


## Truncate and load method


In [62]:
def trunc_and_load(con,data):
    # we simply truncate the table and load the new data
    con.sql(("CREATE OR REPLACE TABLE data_trunc AS SELECT * FROM data"))

with duckdb.connect(file) as con:
    trunc_and_load(con, new_data)
    print(con.sql('SELECT * FROM data_trunc').fetchdf())    

   id     name last_update
0   2    Alice  2021-01-01
1   3  Charlie  2021-01-01
2   4    David  2022-01-01
3   5      Eve  2022-01-01
